<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Init-Dependencies" data-toc-modified-id="Init-Dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Init Dependencies</a></span><ul class="toc-item"><li><span><a href="#Workspace-Setup" data-toc-modified-id="Workspace-Setup-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Workspace Setup</a></span></li></ul></li><li><span><a href="#Set-Static-Variables" data-toc-modified-id="Set-Static-Variables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set Static Variables</a></span></li><li><span><a href="#Set-Authentication" data-toc-modified-id="Set-Authentication-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Set Authentication</a></span></li><li><span><a href="#Helper-Functions" data-toc-modified-id="Helper-Functions-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Helper Functions</a></span></li><li><span><a href="#Begin-Transformations" data-toc-modified-id="Begin-Transformations-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Begin Transformations</a></span></li><li><span><a href="#Update-Result-Framework" data-toc-modified-id="Update-Result-Framework-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Update Result Framework</a></span><ul class="toc-item"><li><span><a href="#API-Response" data-toc-modified-id="API-Response-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>API Response</a></span></li></ul></li><li><span><a href="#Output-Schema" data-toc-modified-id="Output-Schema-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Output Schema</a></span></li></ul></div>

## Init Dependencies

In [1]:
import requests
import os
import pandas as pd
from datetime import datetime

### Workspace Setup

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

## Set Static Variables

In [3]:
URL = 'https://rsr.akvo.org/rest/v1/'
PROJECT_ID = '7283'
RSR_TOKEN = os.environ['RSR_TOKEN']
FMT = '/?format=json&limit=1'
FMT100 = '/?format=json&limit=100'

## Set Authentication

In [4]:
headers = {
    'content-type': 'application/json',
    'Authorization': RSR_TOKEN
}

## Helper Functions

In [5]:
def get_response(endpoint, param, value):
    uri = '{}{}{}&{}={}'.format(URL, endpoint, FMT100, param, value)
    print(get_time() + ' Fetching - ' + uri)
    data = requests.get(uri, headers=headers)
    data = data.json()
    return data

In [6]:
def get_time():
    now = datetime.now().time().strftime("%H:%M:%S")
    return now

In [7]:
def get_sibling_id(x):
    for k,v in x.items():
        return k

In [8]:
def get_report_type(ps,pe):
    rt = {'is_yearly':False}
    psm = ps.split('-')[1]
    pem = pe.split('-')[1]
    if psm == '01' and pem == '12':
        rt = {'is_yearly':True}
    if psm == '01' and pem == '01':
        rt = {'is_yearly':True}
    return rt

In [9]:
def get_dimension_country(dv):
    dp = dv['value'].split(' - ')
    dv = {}
    if dp[0].lower() in ['zambia','malawi','mozambique']:
        dv.update({
            'commodity':'',
            'country':dp[0],
            'has_commodity':False,
            'has_country':True
        })
    else:
        dv.update({
            'commodity':dp[0],
            'country':'',
            'has_commodity':False,
            'has_country':True
        })
    if len(dp) == 2:
        dv.update({
            'commodity':dp[0],
            'country':dp[1],
            'has_commodity':True,
            'has_country':True
        })
    return dv

## Begin Transformations

In [70]:
results_framework = get_response('results_framework','project',PROJECT_ID)['results']

07:44:08 Fetching - https://rsr.akvo.org/rest/v1/results_framework/?format=json&limit=100&project=7283


In [71]:
indicators = []
periods = []
dimension_names = []
dimension_values = []
for result_framework in results_framework:
    rf_id = {'result':result_framework['id']}
    for indicator in result_framework['indicators']:
        indicator_id = indicator['id']
        for period in indicator['periods']:
            is_yearly = get_report_type(period['period_start'],period['period_end'])
            period.update(is_yearly)
            period.update(rf_id)
            period.update({'indicator':indicator_id})
            periods.append(period)
        del indicator['periods']
        for dimension_name in indicator['dimension_names']:
            for dimension_value in dimension_name['values']:
                dimension_value.update(rf_id)
                dimension_update = get_dimension_country(dimension_value)
                dimension_value.update(dimension_update)
                dimension_values.append(dimension_value)
            del dimension_name['values']
            dimension_name.update(rf_id)
            dimension_name.update({'indicator':indicator_id})
            dimension_names.append(dimension_name)
        del indicator['dimension_names']
        indicators.append(indicator)

## Update Result Framework

In [72]:
periods_df = pd.DataFrame(periods)
periods_df = periods_df.groupby(['is_yearly','result']).size().to_frame('size').reset_index().to_dict('records')

In [73]:
reports_annual = []
reports_semester = []
reports_both = []

In [74]:
for period_df in periods_df:
    if period_df['is_yearly']:
        reports_annual.append(period_df['result'])
    else:
        reports_semester.append(period_df['result'])
for y in reports_annual:
    for s in reports_semester:
        if y == s:
            reports_both.append(y)
for m in reports_both:
    reports_annual.remove(m)
    reports_semester.remove(m)

In [75]:
reports_both

[30243, 30307, 30308, 30449]

In [76]:
results_framework_new = []
for rf in results_framework:
    report_type = 'both'
    if rf['id'] in reports_annual:
        report_type = 'annual'
    if rf['id'] in reports_semester:
        report_type = 'semeseter'
    rf.update({'report_type':report_type})
    try:
        child_project = get_sibling_id(rf['child_projects'])
        rf.update({'child_projects': child_project})
    except:
        rf.update({'child_projects': None})
    try:
        parent_project = get_sibling_id(rf['parent_project'])
        rf.update({'parent_project': parent_project})
    except:
        rf.update({'parent_project': None})
    del rf['indicators']
    results_framework_new.append(rf)

### API Response

In [77]:
response = {
    'results_framework':results_framework_new,
    'indicators':indicators,
    'periods':periods,
    'dimension_names':dimension_names,
    'dimension_values':dimension_values
}

In [78]:
response['dimension_names'][0]

{'id': 407,
 'name': 'Type of technology',
 'project': 7283,
 'parent_dimension_name': None,
 'result': 30308,
 'indicator': 47749}

In [79]:
pd.DataFrame(response['results_framework'])

,id,project,project_title,parent_project,child_projects,title,type,aggregation_status,description,order,parent_result,report_type
0,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
1,36599,7283,APPSA Regional - Don't change,None,7950,Intermediate Result 1: Improved collaborative ...,2,None,,1,None,annual
2,36602,7283,APPSA Regional - Don't change,None,7950,Intermediate Result 2: Improved technical capa...,2,None,,2,None,annual
3,36886,7283,APPSA Regional - Don't change,None,7950,Intermediate Result 3: Effective structures an...,2,None,,3,None,annual
4,30307,7283,APPSA Regional - Don't change,None,7282,Promising technologies identified and tested,1,None,,4,None,both
5,30238,7283,APPSA Regional - Don't change,None,7282,"New dissemination packages and tools designed,...",1,None,,5,None,semeseter
6,30243,7283,APPSA Regional - Don't change,None,7282,"Improved regional production, supply and deliv...",2,None,,6,None,both
7,30449,7283,APPSA Regional - Don't change,None,7282,Research institutes are effectively working to...,2,None,,7,None,both
8,30239,7283,APPSA Regional - Don't change,None,None,Improved infrastructure and equipment for targ...,1,None,,8,None,semeseter
9,30423,7283,APPSA Regional - Don't change,None,None,Improved management and performance systems at...,1,None,,9,None,semeseter


In [80]:
d_indicators = pd.DataFrame(response['indicators'])

In [81]:
d_periods = pd.DataFrame(response['periods'])

In [84]:
d_periods['period_time'] = d_periods['period_start'] + ' - ' + d_periods['period_end']

In [94]:
d_periods = d_periods.groupby(['indicator','period_time']).size().to_frame('period_total').reset_index()

In [100]:
d_periods = d_periods.merge(d_indicators,how='inner', left_on='indicator',right_on='id').groupby(['title','period_time','description','indicator']).size().to_frame('total').reset_index()

## Output Schema

In [101]:
d_names = pd.DataFrame(response['dimension_names'])
d_names = d_names.rename(columns={'name':'aggregate'})

In [102]:
d_results = pd.DataFrame(response['results_framework'])

In [103]:
d_values = pd.DataFrame(response['dimension_values']).drop(['result','value','has_commodity','has_country'], axis=1)

In [106]:
d_names = d_names.merge(d_periods, how='inner', left_on='indicator', right_on='indicator')

In [107]:
d_values = d_values.merge(d_names, how='inner', left_on='name', right_on='id')
d_values = d_values.merge(d_results, how='inner', left_on='result', right_on='id')

In [109]:
d_values = d_values.rename(columns={
    'title_x':'indicator_title',
    'title_y': 'title'
})

In [110]:
d_values

,id_x,name,parent_dimension_value,commodity,country,id_y,aggregate,project_x,parent_dimension_name,result,indicator,indicator_title,period_time,description_x,total,id,project_y,project_title,parent_project,child_projects,title,type,aggregation_status,description_y,order,parent_result,report_type
0,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47749,PDO 1 Number of technologies that are being ma...,2013-01-01 - 2013-12-31,"Technology refers to the tools, methods or mac...",1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
1,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47749,PDO 1 Number of technologies that are being ma...,2014-01-01 - 2014-12-31,"Technology refers to the tools, methods or mac...",1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
2,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47749,PDO 1 Number of technologies that are being ma...,2015-01-01 - 2015-12-31,"Technology refers to the tools, methods or mac...",1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
3,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47749,PDO 1 Number of technologies that are being ma...,2016-01-01 - 2016-12-31,"Technology refers to the tools, methods or mac...",1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
4,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47749,PDO 1 Number of technologies that are being ma...,2017-01-01 - 2017-12-31,"Technology refers to the tools, methods or mac...",1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
5,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47749,PDO 1 Number of technologies that are being ma...,2018-01-01 - 2018-12-31,"Technology refers to the tools, methods or mac...",1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
6,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47749,PDO 1 Number of technologies that are being ma...,2019-01-01 - 2020-01-31,"Technology refers to the tools, methods or mac...",1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
7,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47750,PDO 3 Number of technologies generated or prom...,2013-01-01 - 2013-12-31,“Technology generated” - innovation developed....,1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
8,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47750,PDO 3 Number of technologies generated or prom...,2014-01-01 - 2014-12-31,“Technology generated” - innovation developed....,1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both
9,928,407,None,Number of improved seed varieties,,407,Type of technology,7283,None,30308,47750,PDO 3 Number of technologies generated or prom...,2015-01-01 - 2015-12-31,“Technology generated” - innovation developed....,1,30308,7283,APPSA Regional - Don't change,None,7282,Program Development Objective Indicators,2,None,,0,None,both


In [111]:
d_values = d_values.groupby(['title','indicator_title','period_time','aggregate','commodity','country','report_type','child_projects']).size().to_frame('size').sort_index().reset_index()

In [114]:
d_values.groupby(['title','report_type','indicator_title','aggregate','commodity','country','child_projects']).first()

period_time  \
title                                              report_type indicator_title                                    aggregate                         commodity                                          country    child_projects                            
Intermediate Result 1: Improved collaborative t... annual      Number of collaborative research and developmen... Type of research project          Technology dissemination                                      7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Technology generation                                         7950            2013-01-01 - 2013-12-31   
                                                               Percentage of collaborative research and develo... Type of research project          Technology dissemination                                      7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Technology generation                                         7950            2013-01-01 - 2013-12-31   
                                                               Total number of improved technologies formally ... Type of technology                Cassava                                                       7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Legumes                                                       7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Maize                                                         7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Number of improved agronomic, pest and disease,...            7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Number of improved post-harvest storage, labor-...            7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Number of improved seed varieties                             7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Rice                                                          7950            2013-01-01 - 2013-12-31   
                                                               Total number of technologies generated             IRI 7                             Cassava                                                       7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Improved postharvest storage, processing, labor...            7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Legumes                                                       7950            2013-01-01 - 2013-12-31   
                                                                                                                                                    Maize                                     